In [3]:
# Import and set up
from plotly.io import show
from sklearn.model_selection import train_test_split
from skfolio import Population
from skfolio.optimization import (
    EqualWeighted, 
    MaximumDiversification,
    Random
)
from skfolio.preprocessing import prices_to_returns
from openbb import obb


sectors = [
    "XLE", 
    "XLF", 
    "XLU", 
    "XLI", 
    "GDX", 
    "XLK", 
    "XLV", 
    "XLY", 
    "XLP", 
    "XLB", 
    "XOP", 
    "IYR", 
    "XHB", 
    "ITB", 
    "VNQ", 
    "GDXJ", 
    "IYE", 
    "OIH", 
    "XME", 
    "XRT", 
    "SMH", 
    "IBB", 
    "KBE", 
    "KRE", 
    "XTL", 
]

(CVXPY) Jan 21 08:20:37 AM: Encountered unexpected exception importing solver SCS:
ImportError("dlopen(/opt/homebrew/Caskroom/miniforge/base/envs/py311/lib/python3.11/site-packages/_scs_direct.cpython-311-darwin.so, 0x0002): tried: '/opt/homebrew/Caskroom/miniforge/base/envs/py311/lib/python3.11/site-packages/_scs_direct.cpython-311-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/Caskroom/miniforge/base/envs/py311/lib/python3.11/site-packages/_scs_direct.cpython-311-darwin.so' (no such file), '/opt/homebrew/Caskroom/miniforge/base/envs/py311/lib/python3.11/site-packages/_scs_direct.cpython-311-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64'))")
(CVXPY) Jan 21 08:20:38 AM: Encountered unexpected exception importing solver SCS:
ImportError("dlopen(/opt/homebrew/Caskroom/miniforge/base/envs/py311/lib/python3.11/site-packages/_scs_direct.cpython-311-darw

In [4]:
df = obb.equity.price.historical(
    sectors, 
    start_date="2010-01-01", 
    provider="yfinance"
).to_df()

pivoted = df.pivot(
    columns="symbol", 
    values="close"
).dropna()

X = prices_to_returns(pivoted)

X_train, X_test = train_test_split(
    X, 
    test_size=0.33, 
    shuffle=False
)

ValidationError: 1 validation error for historical
provider
  Input should be 'fmp', 'intrinio', 'polygon' or 'tiingo' [type=literal_error, input_value='yfinance', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/literal_error

In [ ]:
# Build the model
model = MaximumDiversification()
model.fit(X_train)
ptf_model_train = model.predict(X_train)

bench = EqualWeighted()
bench.fit(X_train)
ptf_bench_train = bench.predict(X_train)

random = Random()
random.fit(X_train)
ptf_random_train = random.predict(X_train)

print(f"Maximum Diversification: {ptf_model_train.diversification:0.2f}")
print(f"Equal Weighted model: {ptf_bench_train.diversification:0.2f}")
print(f"Random Weighted model: {ptf_random_train.diversification:0.2f}")

In [ ]:
# Predict portfolio weight
ptf_model_test = model.predict(X_test)
ptf_bench_test = bench.predict(X_test)
ptf_random_test = random.predict(X_test)

population = Population([
    ptf_model_test, 
    ptf_bench_test, 
    ptf_random_test
])

population.plot_composition()

In [ ]:
population.plot_cumulative_returns()